Importamos o requests e o bs4 para fazer um requisição HTTP "get" do site e depois fazemos o scrapping do site pegando o conteúdo de interesse

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
def get_poems_authors(url):
    r = requests.get(url)
    print(f"The url: {url} status code it's {r.status_code}")

    html_doc = r.text
    soup = BeautifulSoup(html_doc)

    authors = []
    poems = []

    for span in soup.find_all("span", class_="autor"):
        author = span.text
        authors.append(author)

    for span in soup.find_all("p", class_=re.compile(r"frase\s.*")):
        poem = span.text
        poems.append(poem)

    current_page = soup.find("span", class_="atual")
    next_page = current_page.find_next("a")
    
    href_next_page = next_page['href']

    if 'poemas' not in href_next_page:
        href_next_page = False

    return href_next_page,authors,poems

Em razão da paginação do site, temos que buscar qual seria a próxima página, retornando o href como parâmetro na função acima e fazendo o scrpping da nova página

In [3]:
main_page = "https://www.pensador.com"
poem_page = "https://www.pensador.com/poemas/"
href_exists = True

urls = []

while href_exists:
    href,authors,poems = get_poems_authors(poem_page)

    href_exists = True if href else False
    
    if not href_exists:
        break
    
    poem_page = main_page+href
    urls.append(poem_page)

The url: https://www.pensador.com/poemas/ status code it's 200
The url: https://www.pensador.com/poemas/2/ status code it's 200
The url: https://www.pensador.com/poemas/3/ status code it's 200
The url: https://www.pensador.com/poemas/4/ status code it's 200
The url: https://www.pensador.com/poemas/5/ status code it's 200
The url: https://www.pensador.com/poemas/6/ status code it's 200
The url: https://www.pensador.com/poemas/7/ status code it's 200
The url: https://www.pensador.com/poemas/8/ status code it's 200
The url: https://www.pensador.com/poemas/9/ status code it's 200
The url: https://www.pensador.com/poemas/10/ status code it's 200
The url: https://www.pensador.com/poemas/11/ status code it's 200
The url: https://www.pensador.com/poemas/12/ status code it's 200
The url: https://www.pensador.com/poemas/13/ status code it's 200
The url: https://www.pensador.com/poemas/14/ status code it's 200
The url: https://www.pensador.com/poemas/15/ status code it's 200
The url: https://www.

# Criando um dataset que possui o autor e o seu poema

In [10]:
authors = []
poems = []

for url in urls:
    authors.append(get_poems_authors(url)[1])
    poems.append(get_poems_authors(url)[2])

The url: https://www.pensador.com/poemas/2/ status code it's 200
The url: https://www.pensador.com/poemas/2/ status code it's 200
The url: https://www.pensador.com/poemas/3/ status code it's 200
The url: https://www.pensador.com/poemas/3/ status code it's 200
The url: https://www.pensador.com/poemas/4/ status code it's 200
The url: https://www.pensador.com/poemas/4/ status code it's 200
The url: https://www.pensador.com/poemas/5/ status code it's 200
The url: https://www.pensador.com/poemas/5/ status code it's 200
The url: https://www.pensador.com/poemas/6/ status code it's 200
The url: https://www.pensador.com/poemas/6/ status code it's 200
The url: https://www.pensador.com/poemas/7/ status code it's 200
The url: https://www.pensador.com/poemas/7/ status code it's 200
The url: https://www.pensador.com/poemas/8/ status code it's 200
The url: https://www.pensador.com/poemas/8/ status code it's 200
The url: https://www.pensador.com/poemas/9/ status code it's 200
The url: https://www.pens

In [11]:
len(authors), len(poems)

(46, 46)

In [12]:
def simple_list(_list):
    if isinstance(_list, list):
        return [sub_elem for elem in _list for sub_elem in simple_list(elem)]
    else:
        return [_list]

In [13]:
len(simple_list(authors)), len(simple_list(poems))

(912, 912)

In [39]:
authors = simple_list(authors)
poems = simple_list(poems)

In [48]:
data = pd.DataFrame(list(zip(authors, poems)), columns =['Authors', 'Poems'])

In [49]:
for idx in range(len(data)):
    data['Authors'][idx] = data['Authors'][idx].replace('\n', '').strip()
    data['Poems'][idx] = data['Poems'][idx]

In [50]:
data

,Authors,Poems
0,Octavio Paz,DESTINO DO POETA\n\nPalavras? Sim. De ar\ne pe...
1,Chico Buarque,Mesmo que você fuja de mim \nPor labirintos e ...
2,Bráulio Bessa,"Mulher tu não és igual\ntu não és regra, ou pa..."
3,Jane Austen,Elizabeth: Eu me pergunto quem descobriu o pod...
4,Adélia Prado,"Eu quero uma licença de dormir,\nperdão pra de..."
...,...,...
907,Desconhecido,Eu não definitivamente nesse mundo existo sem ...
908,Thaís Moura,"Esse poema é dedicado a uma grande amiga, a qu..."
909,Poemas e Poesias Catarinenses - Face,A Especial\n\nEu lhe vi longe o bastante para ...
910,Poemas e Poesias Catarinenses - Face,(sem título)\nde: José Ricardo de Matos Pereir...


In [47]:
data.to_csv('../data/authors_poems.csv')